In [ ]:
# default_exp data.core

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# data.core

> This module contains the core bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data in a way modelable by huggingface transformer implementations.

In [ ]:
#export
from functools import reduce

import torch,pdb
from transformers import *
from fastai.text.all import *

from blurr.utils import *

logging.set_verbosity_error()

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.3.1
Using transformers 4.5.1


In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Base tokenization, batch transform, and DataBlock methods

In [ ]:
#export
class HF_BaseInput(TensorBase):  
    
    def show(self, hf_tokenizer, ctx=None, trunc_at=None, **kwargs):
        input_ids = self.cpu().numpy()
        decoded_input = str(hf_tokenizer.decode(input_ids, skip_special_tokens=True))[:trunc_at]

        return show_title(decoded_input, ctx=ctx, label='text')

A `HF_BaseInput` object is returned from the decodes method of `HF_AfterBatchTransform` as a means to customize @typedispatched functions like `DataLoaders.show_batch` and `Learner.show_results`. It uses the "input_ids" of a huggingface object as the representative tensor for `show` methods

In [ ]:
#hide
# played with the idea of writing my own collation routine, but opted to do everything in a batch transform
# and let fastai handle the rest.  keeping this here for the interested :)

# def create_blurr_batch(samples): 
#     """Supports passing in one or two input sequences, or a list[str] (the later is common for token 
#     classification tasks where you should also set `is_split_into_words=True`).
#     Returns all the tensors for the input sequence(s) in a dictionary."""
#     samples = L(samples)
#     inps = samples.itemgot(0).items if (n_seqs == 1 ) else list(zip(samples.itemgot(0,0), samples.itemgot(0,1)))

#     res = hf_tokenizer(inps, max_length=512, padding=True, truncation=True, is_split_into_words=False,return_tensors='pt')

#     fin = (res, retain_type(torch.stack(samples.itemgot(1).items), samples.itemgot(1)[0]))
#     return fin

In [ ]:
#export
class HF_BeforeBatchTransform(Transform):
    """Handles everything you need to assemble a mini-batch of inputs and targets, as well as 
    decode the dictionary produced as a byproduct of the tokenization process in the `encodes` method.
    """
    def __init__(self, hf_arch, hf_config, hf_tokenizer, hf_model,
                 max_length=None, padding=True, truncation=True, is_split_into_words=False, 
                 tok_kwargs={}, **kwargs):

        store_attr(self=self, names='hf_arch, hf_config, hf_tokenizer, hf_model')
        store_attr(self=self, names='max_length, padding, truncation, is_split_into_words, tok_kwargs')
        store_attr(self=self, names='kwargs')

    def encodes(self, samples): 
        samples = L(samples)
        
        # grab inputs
        if (is_listy(samples[0][0]) and not self.is_split_into_words): 
            inps = list(zip(samples.itemgot(0, 0), samples.itemgot(0, 1)))
        else:
            inps = samples.itemgot(0).items
            
        # tokenize
        tok_d = self.hf_tokenizer(inps, 
                                  max_length=self.max_length, 
                                  padding=self.padding, 
                                  truncation=self.truncation, 
                                  is_split_into_words=self.is_split_into_words,
                                  return_tensors='pt', 
                                  **self.tok_kwargs)

        # update samples with tokenized inputs (e.g. input_ids, attention_mask, etc...)
        d_keys = tok_d.keys()
        updated_samples= [ (*[{k: tok_d[k][idx] for k in d_keys}], *sample[1:]) 
                          for idx, sample in enumerate(samples) ]
  
        return updated_samples

`HF_BeforeBatchTransform` was inspired by this [article](http://dev.fast.ai/tutorial.transformers).

Inputs can come in as a string or a list of tokens, the later being for tasks like Named Entity Recognition (NER), where you want to predict the label of each token.

**Notes re: on-the-fly batch-time tokenization**: The previous version of the library performed the tokenization/numericalization as a type transform when the raw data was read, and included a couple batch transforms to prepare the data for collation (e.g., to be made into a mini-batch). With this update, everything is done in a single batch transform.  Why?  Part of the inspiration had to do with the mechanics of the huggingrace tokenizer, in particular how by default it returns a collated mini-batch of data given a list of sequences. And where do we get a list of examples with fastai? In the batch transforms!  So I thought, hey, why not do everything dynamically at batch time?  And with a bit of tweaking, I got everything to work pretty well.  The result is less code, faster mini-batch creation, less RAM utilization and time spent tokenizing (really helps with very large datasets), and more flexibility.

In [ ]:
#export
class HF_AfterBatchTransform(Transform):
    
    def __init__(self, hf_tokenizer, input_return_type=HF_BaseInput):
        store_attr(self=self, names='hf_tokenizer, input_return_type')
        
    def decodes(self, encoded_samples):
        if (isinstance(encoded_samples, dict)): 
            return self.input_return_type(encoded_samples['input_ids'], hf_tokenizer=self.hf_tokenizer)
        return encoded_samples

With fastai 2.1.5, before batch transforms no longer have a `decodes` method ... and so, I've introduced a standard batch transform here, `HF_AfterBatchTransform`, that will do the decoding for us.

In [ ]:
#export
class HF_TextBlock(TransformBlock):
    
    def __init__(self, hf_arch=None, hf_config=None, hf_tokenizer=None, hf_model=None,
                 before_batch_tfm=None, after_batch_tfm=None,
                 max_length=None, padding=True, truncation=True, is_split_into_words=False, 
                 input_return_type=HF_BaseInput, dl_type=SortedDL, 
                 before_batch_kwargs={}, after_batch_kwargs={}, tok_kwargs={}, **kwargs):
        
        if((not all([hf_arch, hf_config, hf_tokenizer, hf_model])) and before_batch_tfm is None):
            raise ValueError("""You must supply the huggingfrace architecture, config, tokenizer, and model
                                - or - an instances of HF_BeforeBatchTransform""")
        
        if (before_batch_tfm is None):
            before_batch_tfm = HF_BeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                       max_length=max_length, 
                                                       padding=padding, 
                                                       truncation=truncation, 
                                                       is_split_into_words=is_split_into_words,
                                                       tok_kwargs=tok_kwargs.copy(), 
                                                       **before_batch_kwargs.copy())
            
        # as we'll have our HF_BeforeBatchTransforme by now, we'll use it to set the TransformBlock's 
        # huggingface face objects
        self.hf_arch = before_batch_tfm.hf_arch
        self.hf_config = before_batch_tfm.hf_config
        self.hf_tokenizer = before_batch_tfm.hf_tokenizer
        self.hf_model = before_batch_tfm.hf_model
            
        if (after_batch_tfm is None): 
            after_batch_tfm = HF_AfterBatchTransform(self.hf_tokenizer, input_return_type, 
                                                     **after_batch_kwargs.copy())
            
        return super().__init__(dl_type=dl_type, 
                                dls_kwargs={ 'before_batch': before_batch_tfm }, 
                                batch_tfms=after_batch_tfm)          

A basic wrapper that links defaults transforms for the data block API

`HF_TextBlock` has been dramatically simplified from it's predecessor. It handles setting up your `HF_BeforeBatchTransform` and `HF_AfterBatchTransform` transforms regardless of data source (e.g., this will work with files, DataFrames, whatever). You must either pass in your own instance of a `HF_BeforeBatchTransform` class or the huggingface architecture and tokenizer via the `hf_arch` and `hf_tokenizer` (the other args are optional).

In [ ]:
#export
def get_blurr_tfm(tfms_list, tfm_class=HF_BeforeBatchTransform):
    return next(filter(lambda el: issubclass(type(el), tfm_class), tfms_list), None)

In [ ]:
#export
@typedispatch
def show_batch(x:HF_BaseInput, y, samples, dataloaders, ctxs=None, max_n=6, trunc_at=None, **kwargs):  
    # grab our tokenizer and pass it and "trunc_at" to HF_BaseInput.show via kwargs
    hf_before_batch_tfm = get_blurr_tfm(dataloaders.before_batch)
    kwargs['hf_tokenizer'] = hf_before_batch_tfm.hf_tokenizer
    kwargs['trunc_at'] = trunc_at
    
    if ctxs is None: ctxs = get_empty_df(min(len(samples), max_n))
    ctxs = show_batch[object](x, y, samples, max_n=max_n, ctxs=ctxs, **kwargs)

    display_df(pd.DataFrame(ctxs))
    return ctxs

## Sequence classification

Below demonstrates how to contruct your `DataBlock` for a sequence classification task (e.g., a model that requires a single text input)

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [ ]:
imdb_df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


There are a bunch of ways we can get at the four huggingface elements we need (e.g., architecture name, tokenizer, config, and model).  We can just create them directly, or we can use one of the helper methods available via `BLURR`.

In [ ]:
#hide_output
model_cls = AutoModelForSequenceClassification

pretrained_model_name = "roberta-base" # "distilbert-base-uncased" "bert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

Once you have those elements, you can create your `DataBlock` as simple as the below.

In [ ]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

In [ ]:
dls = dblock.dataloaders(imdb_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
b = dls.one_batch(); len(b), len(b[0]['input_ids']), b[0]['input_ids'].shape, len(b[1]) 

(2, 4, torch.Size([4, 512]), 4)

Let's take a look at the actual types represented by our batch

In [ ]:
explode_types(b)

{tuple: [dict, fastai.torch_core.TensorCategory]}

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower",negative
1,"I really wanted to love this show. I truly, honestly did.<br /><br />For the first time, gay viewers get their own version of the ""The Bachelor"". With the help of his obligatory ""hag"" Andra, James, a good looking, well-to-do thirty-something has the chance of love with 15 suitors (or ""mates"" as they are referred to in the show). The only problem is half of them are straight and James doesn't know this. If James picks a gay one, they get a trip to New Zealand, and If he picks a straight one, str",negative


## Tests

The tests below to ensure the core DataBlock code above works for **all** pretrained sequence classification models available in huggingface.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained classification models you are working with ... and if any of your pretrained sequence classification models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
[ model_type for model_type in BLURR.get_models(task='SequenceClassification') 
 if (not model_type.__name__.startswith('TF')) ]

[transformers.models.albert.modeling_albert.AlbertForSequenceClassification,
 transformers.models.bart.modeling_bart.BartForSequenceClassification,
 transformers.models.bert.modeling_bert.BertForSequenceClassification,
 transformers.models.big_bird.modeling_big_bird.BigBirdForSequenceClassification,
 transformers.models.ctrl.modeling_ctrl.CTRLForSequenceClassification,
 transformers.models.camembert.modeling_camembert.CamembertForSequenceClassification,
 transformers.models.convbert.modeling_convbert.ConvBertForSequenceClassification,
 transformers.models.deberta.modeling_deberta.DebertaForSequenceClassification,
 transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification,
 transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification,
 transformers.models.electra.modeling_electra.ElectraForSequenceClassification,
 transformers.models.flaubert.modeling_flaubert.FlaubertForSequenceClassification,
 transformers.models.funnel.modeling_fu

In [ ]:
pretrained_model_names = [
    'albert-base-v1',
    'facebook/bart-base',
    'bert-base-uncased',
    'google/bigbird-roberta-base',
    'sshleifer/tiny-ctrl',
    'camembert-base',
    'sarnikowski/convbert-medium-small-da-cased',
    'microsoft/deberta-base',
    'microsoft/deberta-v2-xlarge',
    'distilbert-base-uncased',
    'monologg/electra-small-finetuned-imdb',
    'flaubert/flaubert_small_cased', 
    'huggingface/funnel-small-base',
    'gpt2',
    'kssteven/ibert-roberta-base',
    'allenai/led-base-16384',
    'microsoft/layoutlm-base-uncased',
    'allenai/longformer-base-4096',
    'sshleifer/tiny-mbart', 
    'microsoft/mpnet-base',
    'google/mobilebert-uncased',
    'openai-gpt',
    #'reformer-enwik8',                  # (see model card; does not work with/require a tokenizer so no bueno here)
    'roberta-base',
    'squeezebert/squeezebert-uncased',
    #'google/tapas-base',                # (requires pip install torch-scatter)
    'transfo-xl-wt103', 
    'xlm-mlm-en-2048',
    'xlm-roberta-base',
    'xlnet-base-cased'
]

In [ ]:
#hide
# for model_name in pretrained_model_names:
#     tok = AutoTokenizer.from_pretrained(model_name)
#     print(f'=== {model_name} ===')
#     print(f'=== {tok.padding_side} ===')
#     print(f'=== {tok.pad_token_id} ===')
#     print(tok(['hi', 'hello everyone. its good to be here'], ['yo', 'yo'], padding='max_length', max_length=128))

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [ ]:
#slow
#hide_output
model_cls = AutoModelForSequenceClassification
bsz = 2
seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    tok_class = RobertaTokenizer if ('/ibert' in model_name) else None

    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, 
                                                                      model_cls=model_cls, 
                                                                      tokenizer_cls=tok_class)   
    
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n')
    
    # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
    if (hf_tokenizer.pad_token is None): 
        hf_tokenizer.add_special_tokens({'pad_token': '<pad>'})  
        hf_config.pad_token_id = hf_tokenizer.get_vocab()['<pad>']
        hf_model.resize_token_embeddings(len(hf_tokenizer))   
    
    blocks = (
        HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model, padding='max_length', max_length=seq_sz), 
        CategoryBlock
    )

    dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())
    dls = dblock.dataloaders(imdb_df, bs=bsz) 
    b = dls.one_batch()
    
    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)

        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'PASSED', ''))
        dls.show_batch(dataloaders=dls, max_n=2, trunc_at=1000)
        
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'FAILED', err))

=== albert-base-v1 ===

architecture:	albert
tokenizer:	AlbertTokenizerFast

*** TESTING DataLoaders ***



,text,category
0,"raising victor vargas: a reviewbr /br /you know, raising victor vargas is like sticking your hands into a big, steaming bowl of oatmeal. it's warm and gooey, but you're not sure if it feels right. try as i might, no matter how warm and gooey raising victor vargas became i was always aware that something didn't quite feel right. victor vargas suffers from a certain overconfidence on the director's part. apparently, the director thought that the ethnic backdrop of a latino family on the lower east side, and an i",negative
1,"""the true story of the friendship that shook south africa and awakened the world."" br /br /richard attenborough, who directed ""a bridge too far"" and ""gandhi"", wanted to bring the story of steve biko to life, and the journey and trouble that journalist donald woods went through in order to get his story told. the films uses wood's two books for it's information and basis - ""biko"" and ""asking for trouble"".br /br /the film takes place in the",positive


=== facebook/bart-base ===

architecture:	bart
tokenizer:	BartTokenizerFast

*** TESTING DataLoaders ***



,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic",negative
1,"I really wanted to love this show. I truly, honestly did.<br /><br />For the first time, gay viewers get their own version of the ""The Bachelor"". With the help of his obligatory ""hag"" Andra, James, a good looking, well-to-do thirty-something has the chance of love with 15 suitors (or ""mates"" as they are referred to in the show). The only problem is half of them are straight and James doesn't know this. If James picks a gay one, they get a trip to New Zealand, and If he picks a straight one, straight",negative


=== bert-base-uncased ===

architecture:	bert
tokenizer:	BertTokenizerFast

*** TESTING DataLoaders ***



...

,text,category
0,"raising victor vargas : a review < br / > < br / > you know, raising victor vargas is like sticking your hands into a big, steaming bowl of oatmeal. it's warm and gooey, but you're not sure if it feels right. try as i might, no matter how warm and gooey raising victor vargas became i was always aware that something didn 't quite feel right. victor vargas suffers from a certain overconfidence on the director's part. apparently, the director thought that the ethnic backdrop of a latino family on the lower east side, and an idyllic storyline would make the film critic proof",negative
1,"the shop around the corner is one of the sweetest and most feel-good romantic comedies ever made. there's just no getting around that, and it's hard to actually put one's feeling for this film into words. it's not one of those films that tries too hard, nor does it come up with the oddest possible scenarios to get the two protagonists together in the end. in fact, all its charm is innate, contained within the characters and the setting and the plot... which is highly believable to boot. it's easy to think that such a love story, as beautiful as any other ever told,",positive


=== xlm-roberta-base ===

architecture:	xlm_roberta
tokenizer:	XLMRobertaTokenizerFast

*** TESTING DataLoaders ***



,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic back",negative
1,"I really wanted to love this show. I truly, honestly did.<br /><br />For the first time, gay viewers get their own version of the ""The Bachelor"". With the help of his obligatory ""hag"" Andra, James, a good looking, well-to-do thirty-something has the chance of love with 15 suitors (or ""mates"" as they are referred to in the show). The only problem is half of them are straight and James doesn't know this. If James picks a gay one, they get a trip to New Zealand, and If",negative


=== xlnet-base-cased ===

architecture:	xlnet
tokenizer:	XLNetTokenizerFast

*** TESTING DataLoaders ***



,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east",negative
1,"Many neglect that this isn't just a classic due to the fact that it's the first 3D game, or even the first shoot-'em-up. It's also one of the first stealth games, one of the only(and definitely the first) truly claustrophobic games, and just a pretty well-rounded gaming experience in general. With graphics that are terribly dated today, the game thrusts you into the role of B.J.(don't even *think* I'm going to attempt spelling his last name!), an American P.O.W",positive


In [ ]:
#slow
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,albert,AlbertTokenizerFast,albert-base-v1,PASSED,
1,bart,BartTokenizerFast,facebook/bart-base,PASSED,
2,bert,BertTokenizerFast,bert-base-uncased,PASSED,
3,big_bird,BigBirdTokenizer,google/bigbird-roberta-base,PASSED,
4,ctrl,CTRLTokenizer,sshleifer/tiny-ctrl,PASSED,
5,camembert,CamembertTokenizerFast,camembert-base,PASSED,
6,convbert,ConvBertTokenizerFast,sarnikowski/convbert-medium-small-da-cased,PASSED,
7,deberta,DebertaTokenizer,microsoft/deberta-base,PASSED,
8,deberta_v2,DebertaV2Tokenizer,microsoft/deberta-v2-xlarge,PASSED,
9,distilbert,DistilBertTokenizerFast,distilbert-base-uncased,PASSED,


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-token-classification.ipynb.
Converted 01b_data-question-answering.ipynb.
Converted 01za_data-seq2seq-core.ipynb.
Converted 01zb_data-seq2seq-language-modeling.ipynb.
Converted 01zc_data-seq2seq-summarization.ipynb.
Converted 01zd_data-seq2seq-translation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-token-classification.ipynb.
Converted 02b_modeling-question-answering.ipynb.
Converted 02za_modeling-seq2seq-core.ipynb.
Converted 02zb_modeling-seq2seq-language-modeling.ipynb.
Converted 02zc_modeling-seq2seq-summarization.ipynb.
Converted 02zc_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-multilabel.ipynb.
Converted index.ipynb.
